
# **Entrega Final**

**Datasets:** `ventas.csv`, `clientes.csv`, `marketing.csv`

1. Entorno & carga de datos ✔️  
4. Exploración inicial con Pandas ✔️  
5. Calidad de datos ✔️  
6. Limpieza del dataset ✔️


## 1) Entorno y **carga de datos**

In [ ]:
# Importar la librería
import pandas as pd

In [ ]:
# archivos de Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Muestra que archivos hay en la carpeta donde están los datos
import os
print("Los archivos en la carpeta de datos son:")
print(os.listdir("/content/drive/MyDrive/datasets"))

In [ ]:
# rutas nuestros archivos CSV
ruta_ventas = "/content/drive/MyDrive/datasets/ventas.csv"
ruta_clientes = "/content/drive/MyDrive/datasets/clientes.csv"
ruta_marketing = "/content/drive/MyDrive/datasets/marketing.csv"

# Generamos los DataFrames a partir de los archivos csv
mis_ventas = pd.read_csv(ruta_ventas)
mis_clientes = pd.read_csv(ruta_clientes)
mi_marketing = pd.read_csv(ruta_marketing)

# filas y columnas tiene cada uno
print("\nCantidad de filas y columnas:")
print("Ventas:", mis_ventas.shape)
print("Clientes:", mis_clientes.shape)
print("Marketing:", mi_marketing.shape)

# primeras filas para saber cómo son los datos
print("\nAsí se ven las primeras filas:")
display(mis_ventas.head())
display(mis_clientes.head())
display(mi_marketing.head())

## 4) **Exploración inicial con pandas**

In [ ]:
# función para ver información básica del dataframe
def ver_info(df, nombre):
    print(f"--- Información de {nombre} ---")
    print("Forma (filas, columnas):", df.shape)
    print("Columnas:", list(df.columns))
    print("Tipos de datos:")
    print(df.dtypes)
    print("\n¿Cuántos datos faltan en cada columna?")
    print(df.isna().sum())
    print("\nAlgunas filas para ver cómo son:")
    display(df.head())
    print("\nEstadísticas de los números:")
    display(df.describe())
    print("-" * 30)

In [ ]:
# Usamos la función para ver la info de las ventas
ver_info(mis_ventas, "mis ventas")

In [ ]:
# Vemos la info de los clientes
ver_info(mis_clientes, "mis clientes")

In [ ]:
# la info del marketing
ver_info(mi_marketing, "mi marketing")

## 5) **Calidad de datos** (nulos y duplicados)

In [ ]:
# Función para revisar si hay datos que se repiten o faltan
def revisar_calidad(df, nombre, columna_clave=None):
    print(f"--- Revisando {nombre} ---")

    # Miramos cuántos datos faltan
    print("Datos que faltan en cada columna:")
    display(df.isna().sum().to_frame("faltan"))

    # Vemos si hay filas completas que están repetidas
    filas_repetidas = df.duplicated(keep=False).sum()
    print(f"Total de filas repetidas: {filas_repetidas}")

    # Vemos si hay repetidos por columna clave
    if columna_clave and columna_clave in df.columns:
        repetidos_clave = df[columna_clave].duplicated(keep=False).sum()
        print(f"Valores repetidos en la columna '{columna_clave}': {repetidos_clave}")

        # Si hay repetidos en esa columna, mostramos cuáles son los que más se repiten
        if repetidos_clave > 0:
            valores_repetidos_clave = df[df[columna_clave].duplicated(keep=False)][columna_clave].value_counts().head(10)
            print("\nLos valores que más se repiten en esa columna son:")
            display(valores_repetidos_clave)
        else:
            print(f"No hay valores repetidos en la columna '{columna_clave}'.")
    else:
        if columna_clave:
            print(f"¡Ojo! La columna '{columna_clave}' no existe en este dataset.")
        else:
            print("No revisamos duplicados por una columna específica.")
    print("-" * 30)

In [ ]:
# Las ventas, usando el id de venta como columna clave
revisar_calidad(mis_ventas, "mis ventas", columna_clave="id_venta")

In [ ]:
# Clientes, usando el id de cliente como columna clave
revisar_calidad(mis_clientes, "mis clientes", columna_clave="id_cliente")

In [ ]:
# Marketing con el id de campaña
revisar_calidad(mi_marketing, "mi marketing", columna_clave="id_campanha")


## 6) **Limpieza del dataset**
- Eliminamos duplicados.
- Normalizamos **texto** en columnas `object` (trim + capitalización simple).
- Convertimos fechas a fechas reales
- Convertimos `precio` y `cantidad` a numéricos si existen.
- Guardamos CSV limpios.


In [ ]:
# Creamos copias de nuestros datos para no dañar los originales mientras limpiamos
ventas_limpias = mis_ventas.copy()
clientes_limpios = mis_clientes.copy()
marketing_limpio = mi_marketing.copy()

# Quitamos las filas que son exactamente iguales
ventas_limpias = ventas_limpias.drop_duplicates()
clientes_limpios = clientes_limpios.drop_duplicates()
marketing_limpio = marketing_limpio.drop_duplicates()

In [ ]:
# Una función para arreglar el texto en las columnas de tipo string
def arreglar_texto(df):
    # Vamos columna por columna
    for columna in df.select_dtypes(include="object").columns:
        # Convertimos a texto, quitamos espacios al principio y final,
        # quitamos cosas raras como tabulaciones, quitamos espacios extra en medio,
        # y ponemos la primera letra en mayúscula (como un título).
        df[columna] = (
            df[columna]
            .astype(str)
            .str.strip()
            .str.replace(r"[\u200b\t\r\n]", "", regex=True)
            .str.replace(" +", " ", regex=True)
            .str.title()
        )
    return df

In [ ]:
# Convertimos las columnas que parecen fechas en fechas de verdad
for mi_df in [ventas_limpias, clientes_limpios, marketing_limpio]:
    for columna in mi_df.columns:
        # Si el nombre de la columna tiene "fecha"
        if "fecha" in columna.lower():
            # Intentamos convertirla a fecha. Si no puede, pone un valor especial (NaT).
            # dayfirst=True ayuda a entender fechas como "día/mes/año".
            mi_df[columna] = pd.to_datetime(mi_df[columna], errors="coerce", dayfirst=True)

In [ ]:
# Aplicamos la función para arreglar el texto a nuestros dataframes limpios
ventas_limpias = arreglar_texto(ventas_limpias)
clientes_limpios = arreglar_texto(clientes_limpios)
marketing_limpio = arreglar_texto(marketing_limpio)

In [ ]:
# Ahora arreglamos la columna de precios
# Si existe la columna 'precio' en las ventas limpias
if "precio" in ventas_limpias.columns:
    # Quitamos el signo '$' y las comas, y convertimos a número.
    # Si algo no se puede convertir, se pone como "NaN" (No es un Número).
    ventas_limpias["precio"] = (
        ventas_limpias["precio"]
        .astype(str)
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.strip()
    )
    ventas_limpias["precio"] = pd.to_numeric(ventas_limpias["precio"], errors="coerce")

In [ ]:
# Arreglamos la columna de cantidad
# Si existe la columna 'cantidad' en las ventas limpias
if "cantidad" in ventas_limpias.columns:
    # Convertimos a número. Los errores se convierten en NaN.
    # Usamos "Int64" para que acepte números enteros y también NaN.
    ventas_limpias["cantidad"] = pd.to_numeric(
        ventas_limpias["cantidad"], errors="coerce"
    ).astype("Int64")

In [ ]:
# Vemos un resumen de las ventas limpias para ver si quedó bien
ventas_limpias.info()

# Guardamos los dataframes limpios en nuevos archivos CSV
ventas_limpias.to_csv("ventas_limpias.csv", index=False)
clientes_limpios.to_csv("clientes_limpios.csv", index=False)
marketing_limpio.to_csv("marketing_limpio.csv", index=False)

print("\n¡Listo! Guardamos los archivos limpios:")
print("ventas_limpias.csv")
print("clientes_limpios.csv")
print("marketing_limpio.csv")

## 7) **Métrica de Ingreso**

In [ ]:
# 1. Queremos calcular cuánto dinero (ingreso) ganamos por cada venta.
#    Para esto, necesitamos multiplicar el 'precio' de lo que vendimos por la 'cantidad'.

# Paso 1: Creamos una nueva columna llamada "ingreso" en nuestra tabla 'ventas_clean'.
#         Esto lo hacemos usando corchetes [] y el signo de igual =.
#         Es como crear una columna nueva en una hoja de cálculo.
ventas_limpias["ingreso"] = ventas_limpias["precio"] * ventas_limpias["cantidad"]

# Paso 2: Ahora, guardamos la tabla 'ventas_clean' (que ya tiene la columna 'ingreso')
#         en una variable nueva llamada 'ventas_perf'.
#         Esto es solo para darle un nombre que indique que la tabla está 'perfecta' o 'completa'.
ventas_perf = ventas_limpias

# ¡Listo! La tabla 'ventas_perf' ahora tiene una columna 'ingreso' con el cálculo correcto.

In [ ]:
# 1. Agrupamos los datos de ventas_perf por la columna 'producto'
#    Esto crea grupos separados para cada producto único.

resumen_prod = (
    ventas_perf
    .groupby("producto")

    # 2. Calculamos las métricas para cada grupo:
    .agg(
        # Suma de la columna 'ingreso' para obtener el ingreso total
        ingreso_total=('ingreso', 'sum'),

        # Suma de la columna 'cantidad' para obtener el total de unidades vendidas
        unidades=('cantidad', 'sum'),

        # Promedio de la columna 'precio' para obtener el precio promedio
        precio_promedio=('precio', 'mean'),

        # Cuenta cuántas filas (registros) hay en cada grupo de productos
        registros=('ingreso', 'size')
    )
)

# El resultado 'resumen_prod' es una tabla con las métricas por cada producto.

In [ ]:
print(resumen_prod.head(60))

## Combinacion de set de datos

In [ ]:
#agrupo ventas clean por producto y sum la cantidad por el precio
df_ventas_x_producto = ventas_limpias.groupby('producto').agg(
    precio_por_cantidad=('precio', lambda x: (x * ventas_limpias.loc[x.index, 'cantidad']).sum())
)

# Ordeno por producto de forma ascendente
df_ventas_x_producto = df_ventas_x_producto.sort_values(by='producto', ascending=True)

# muestro resultado
display(df_ventas_x_producto)

In [ ]:
#agrupo marketing clean por producto y sumo el costo de las campañas de ese producto

df_marketing_x_producto = marketing_limpio.groupby('producto')['costo'].sum().reset_index()
df_marketing_x_producto = df_marketing_x_producto[['producto', 'costo']] # Select only the desired columns
df_marketing_x_producto = df_marketing_x_producto.sort_values(by='producto', ascending=True) # Sort by product ascending
display(df_marketing_x_producto)

In [ ]:
#mergeo el df agrupado por producto con el de marketing agrupado por producto por producto
merged_df = pd.merge(df_ventas_x_producto, df_marketing_x_producto, on='producto')
display(merged_df)

In [ ]:
#obtengo que % me representa el costo sobre el ingreso
merged_df['porcentaje_costo_sobre_ingreso'] = (merged_df['costo'] / merged_df['precio_por_cantidad']) * 100
display(merged_df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Boxplot to identify dispersion and potential outliers
plt.figure(figsize=(8, 6))
sns.boxplot(y=merged_df['porcentaje_costo_sobre_ingreso'])
plt.title('Boxplot del Porcentaje de Costo sobre Ingreso')
plt.ylabel('Porcentaje de Costo sobre Ingreso')
plt.show()


## Analisis exploratorio a traves de visualizaciones

In [ ]:
import matplotlib.pyplot as plt  # importamos matplotlib

vals = resumen_prod['ingreso_total'].dropna()  # tomamos valores numéricos y quitamos NaN

In [ ]:
# Histogram to see the distribution
plt.figure(figsize=(8, 6))
sns.histplot(vals, kde=True)
plt.title('Distribución del Ingreso total por producto')
plt.xlabel('Ingreso Total')
plt.ylabel('Frecuencia')
plt.show()